In [3]:
import re
import requests
from bs4 import BeautifulSoup

In [4]:
request_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2022-01'
post_data['gender'] = 'all'

In [5]:
def requestPageText(pageNo : int) :
    if pageNo > 0 :
        post_data['page'] = pageNo
    else :
        raise ValueError('page should be positive value')
    
    res = requests.post(request_url, data=post_data)
    return res.text

In [18]:
def getActorDetailInfoFromPage(actor) :
    # 이름 parsing
    actor_info_dict = dict()
    actor_name = re.sub('\(\w*\)', '', actor.text)
    actor_info_dict['이름'] = actor_name
    
    # 상세페이지 crawling
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    # 상세페이지 데이터에서 배우 상세정보 추출
    for actor_detail in actor_details :
        actor_detail_key = actor_detail.select_one('span.tit').text
        actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail))
        actor_detail_value = re.sub('<.*?>', '', actor_detail_value)
        actor_info_dict[actor_detail_key] = actor_detail_value
    return actor_info_dict
        
    

In [23]:
def parseMovieListFromList(movies) :
    movie_list = list()
    for movie in movies :
        movie_list.append(movie.text)
    return movie_list

In [25]:
actors_info_list = list()

for pageNo in range(1, 11):
    pageText = requestPageText(pageNo)
    soup = BeautifulSoup(pageText, 'html.parser')
    
    actors = soup.select('li.people_li')
    
    for index, actor in enumerate(actors) :
        detail_info = getActorDetailInfoFromPage(actor.select('div.name')[0])
        popularity_index = actor.select('ul.num_info > li > strong')[0].text
        detail_info['흥행지수'] = popularity_index
        movie_list = parseMovieListFromList(actor.select('ul.mov_list > li > a > span'))
        detail_info['출연영화'] = movie_list
        actors_info_list.append(detail_info)

In [26]:
actors_info_list

[{'이름': '강하늘',
  '다른 이름': '김하늘',
  '직업': '배우',
  '생년월일': '1990-02-21',
  '성별': '남',
  '홈페이지': '\nhttp://weibo.com/galpos3?is_hot=1\n',
  '신장/체중': '181cm, 70kg',
  '학교': '중앙대학교 연극학과',
  '흥행지수': '79,392',
  '출연영화': ['해적: 도깨비 깃발', '해피 뉴 이어']},
 {'이름': '한효주',
  '직업': '배우',
  '생년월일': '1987-02-22',
  '성별': '여',
  '홈페이지': '\nhttps://www.facebook.com/hhj.official\n',
  '신장/체중': '170cm',
  '학교': '동국대학교 연극영화',
  '취미': '영화감상',
  '흥행지수': '67,602',
  '출연영화': ['해적: 도깨비 깃발']},
 {'이름': '이광수',
  '직업': '배우',
  '생년월일': '1985-07-14',
  '성별': '남',
  '홈페이지': '\nhttps://twitter.com/masijacoke85\nhttps://www.instagram.com/masijacoke850714/\n',
  '신장/체중': '190cm',
  '소속사': '킹콩엔터테인먼트',
  '흥행지수': '59,677',
  '출연영화': ['해적: 도깨비 깃발', '해피 뉴 이어']},
 {'이름': '권상우',
  '다른 이름': 'Kwon Sang Woo',
  '직업': '배우',
  '생년월일': '1976-08-05',
  '성별': '남',
  '신장/체중': '183cm, 72kg',
  '학교': '한남대학교 미술교육학 학사',
  '취미': '수영, 헬스, 복싱',
  '특기': '농구',
  '소속사': '벨액터스 엔터테인먼트',
  '흥행지수': '48,287',
  '출연영화': ['해적: 도깨비 깃발']},
 {'이름': '조진웅',
  '다른